In [88]:
import numpy as np
import pandas as pd
import nltk 
import gensim
import re

In [89]:
df_quest=pd.read_csv("/Data/Questions.csv",encoding='latin1')
df_t=pd.read_csv("/Data/Tags.csv",encoding='latin1')

In [90]:
df_question=df_quest.iloc[:1500]
#df_tags=df_t.iloc[:800]
df_question.shape

(1500, 7)

In [91]:
df_question.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [92]:
df_mod_question=df_question.drop(['Id','OwnerUserId','CreationDate','Score'],axis=1)
df_mod_question.head()

,ClosedDate,Title,Body
0,NaN,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,2012-12-26T03:45:49Z,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,NaN,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,NaN,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,NaN,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [93]:
titles=df_mod_question.Title
questions=df_mod_question.Body

In [94]:
import string

#remove html characters
def html_using_re(text):
    return(re.sub("(\<.*?\>)","",text))

#removing punctuation using string library
def rem_punctuation(text):
    removeStr=set(string.punctuation)
    return ''.join(char for char in text if char not in removeStr)


questions = [html_using_re(question) for question in questions]
titles = [rem_punctuation(title) for title in titles]
questions = [rem_punctuation(question) for question in questions]

In [95]:
#tokenize word using nltk word_tokenize
questions_token = [nltk.word_tokenize(question) for question in questions]
titles_token = [nltk.word_tokenize(title) for title in titles]

In [96]:
#getting root words
stemmer=nltk.stem.PorterStemmer()
questions_stemmed = [[stemmer.stem(token) for token in tokens] for tokens in questions_token]
titles_stemmed = [[stemmer.stem(token) for token in tokens] for tokens in titles_token]

In [110]:
#removing stopwords:nltk.corpus.stopwords.words

stopwords_english = set([word for word in nltk.corpus.stopwords.words("english")])
questions_stemmed = [[token for token in tokens if token.lower() not in stopwords_english] for tokens in questions_stemmed]
titles_stemmed = [[token for token in tokens if token.lower() not in stopwords_english] for tokens in titles_stemmed]


In [111]:
#word count: count frequecy of words in the doc

count_words = pd.Series(np.concatenate(questions_stemmed)).value_counts()
singular_words = set(count_words[pd.Series(np.concatenate(questions_stemmed)).value_counts() == 1].index)
questions_stemmed = [[word for word in title if word not in singular_words] for title in questions_stemmed]

count_words = pd.Series(np.concatenate(titles_stemmed)).value_counts()
singular_words = set(count_words[pd.Series(np.concatenate(titles_stemmed)).value_counts() == 1].index)
titles_stemmed = [[word for word in title if word not in singular_words] for title in titles_stemmed]


In [112]:
#form list of array of each sentence
non_empty_indices = [i for i in range(len(questions_stemmed)) if len(questions_stemmed[i]) > 0]
questions_stemmed = np.asarray(questions_stemmed)[non_empty_indices]
non_empty_indices = [i for i in range(len(titles_stemmed)) if len(titles_stemmed[i]) > 0]
titles_stemmed = np.asarray(titles_stemmed)[non_empty_indices]

In [113]:
#gensim.corpora.Dictionary- mapping between normalize words and their integer id's(token_id,word)
dictionary = gensim.corpora.Dictionary(questions_stemmed)
dictionary2 = gensim.corpora.Dictionary(titles_stemmed)
print(dictionary2)

Dictionary(961 unique tokens: ['multipl', 'one', 'queri', 'statement', 'branch']...)


In [114]:
#Convert list of wo into bag-of-words(token_id,token_count)
doc2bow = [dictionary.doc2bow(text) for text in questions_stemmed]
doc2bow1 = [dictionary2.doc2bow(text) for text in titles_stemmed]

In [115]:
#Build model using LDA algo
model =  gensim.models.ldamodel.LdaModel
ldamodel = model(doc2bow1, num_topics=5, id2word = dictionary2, passes=50)

In [116]:
print(ldamodel.print_topics(num_topics=5,num_words=10))

[(0, '0.053*"C" + 0.023*"use" + 0.019*"string" + 0.017*"aspnet" + 0.015*"vs" + 0.014*"array" + 0.013*"variabl" + 0.011*"function" + 0.010*"email" + 0.010*"whi"'), (1, '0.036*"use" + 0.024*"control" + 0.020*"file" + 0.020*"net" + 0.017*"error" + 0.016*"web" + 0.014*"build" + 0.012*"aspnet" + 0.012*"applic" + 0.011*"tool"'), (2, '0.041*"way" + 0.038*"best" + 0.022*"get" + 0.018*"file" + 0.012*"C" + 0.011*"sourc" + 0.011*"code" + 0.010*"applic" + 0.010*"eclips" + 0.010*"gener"'), (3, '0.020*"visual" + 0.020*"work" + 0.019*"studio" + 0.017*"test" + 0.016*"make" + 0.015*"object" + 0.014*"differ" + 0.012*"java" + 0.012*"page" + 0.011*"net"'), (4, '0.042*"server" + 0.038*"sql" + 0.027*"use" + 0.021*"data" + 0.019*"databas" + 0.018*"python" + 0.016*"window" + 0.014*"store" + 0.013*"applic" + 0.011*"get"')]


In [117]:
#tf=Term frequency,idf=Inverse document frequency
tfidf = gensim.models.tfidfmodel.TfidfModel(doc2bow1)
doc2bow1_tfidf = tfidf[doc2bow1]
d = {dictionary.get(id): value for doc in doc2bow1_tfidf for id, value in doc}

In [118]:
print(d)

{'adob': 0.5275734756011512, 'air': 0.28595507327358466, 'applic': 0.5772957326124676, 'boolean': 0.3440574205214579, 'call': 0.5864930429570165, 'catch': 0.2205523250169085, 'categoryid': 0.27593444073702883, 'creat': 0.42525269300834995, 'databas': 0.5723266862453216, 'date': 0.18210609353373264, 'durat': 0.5479245837375005, 'error': 0.479474498194099, 'execut': 0.40954890857403414, 'exist': 0.4099984392173051, 'fail': 0.7265557339599586, 'filenam': 0.3902844259798923, 'filestream': 0.22474326300802736, 'first': 0.3096306671799468, 'follow': 0.38589792142793106, 'function': 0.6299021715984993, 'gener': 0.404565780653228, 'howev': 0.3352043764893029, 'integ': 0.27283883303330725, 'ive': 0.243953732714313, 'key': 0.6317530362997883, 'look': 0.30337700929046024, 'method': 0.32288821101420645, 'multipl': 0.4614366613289099, 'new': 0.37587166743491196, 'one': 0.34608356175570987, 'onli': 0.4722325377340358, 'primari': 0.3966830028114962, 'public': 0.43600430132222867, 'queri': 0.449660715